# Song recommender project #

 Garcinuño Feliciano Angela <br>
 Huima Klaara <br>
 Massot Lucas 

In [ ]:
import pandas as pd
import numpy as np
import build
import random
import buildEncoder

In [ ]:
df, term_index = buildEncoder.load_data('output.csv')

In [ ]:
def recommend(song_list: list, df: pd.core.frame.DataFrame, k: int, encoder, term_index, hot_data, distance_matrix = None):
    #print(song_list)
    playlist, indexes = buildEncoder.format_input(song_list, df)
    print("input formatted")
    if isinstance(distance_matrix, type(None)):
        distances, total_d = buildEncoder.compute_distance_to_playlist(playlist, indexes, hot_data, encoder)
    else:
        print("computing distances to playlist...")
        distances, total_d = buildEncoder.select_distance_to_playlist(playlist, indexes, distance_matrix)    
    print("distances computed")
    display(pd.DataFrame(distances))
    output_df = buildEncoder.get_k_recommendations(total_d, k, df)
    print("received recommendations")
    display(output_df)
    return distances, total_d

In [ ]:
hot_data, encoder = buildEncoder.build_encoder(df, term_index, tuning = False)

"""
playlists = []
indexes = []
for i in range(10):
    index = [i*1000 + j for j in range(1000)]
    playlist = [hot_data[i*1000 + j] for j in range(1000)]
    playlists.append(playlist)
    indexes.append(index)
"""

In [ ]:
# TEST
n = 1
#input = [[str(data.iloc[j]['title']), str(data.iloc[j]['artist_name'])] for j in random.sample(range(len(data)), n)]   # input of song titles
songs= [4292,5965,7390]
unsorted_df = build.load_data("output.csv")
input = [[str(unsorted_df.iloc[i]['title']), str(unsorted_df.iloc[i]['artist_name']),str(unsorted_df.iloc[i]['release'])] for i in songs]
k = 10 #number of song recomendations
# Computing complete distance matrix

display(pd.DataFrame(input))
#distance_matrix = pd.DataFrame(build.build_distances(data))
#recommend(input, data, k, distance_matrix)

print("encoder is done")
# Computing only distance to playlist
distances, total_d = recommend(input, df.copy(), k, encoder, term_index, hot_data)

In [ ]:
display(sorted(total_d))